### **Cuaderno: Módulo 1 - Dominando Common Table Expressions (CTEs)**


**Objetivo del Módulo:** Al finalizar este cuaderno, el participante podrá refactorizar consultas complejas, transformándolas en CTEs limpios y modulares. Entenderá cómo esta técnica mejora la legibilidad, facilita la depuración y previene errores comunes, sentando las bases para escribir SQL de nivel profesional.

-----

### **Introducción al Módulo**

En este primer módulo, se aborda un desafío común en SQL: el "infierno de las subconsultas anidadas". Se presentará una técnica fundamental del SQL moderno, las **Common Table Expressions (CTEs)**, para transformar consultas complejas y difíciles de mantener en una lógica de negocio clara, secuencial y robusta. El objetivo es que el participante aprenda a estructurar su pensamiento y su código de una manera más profesional y escalable.

-----

### **El Escenario: Preparación de los Datos**

Todo análisis comienza con los datos. El primer paso consiste en simular un ecosistema de e-commerce realista. El siguiente script de Python genera un conjunto de DataFrames en memoria que representan las entidades de negocio: usuarios, productos y pedidos.

**Acción:** Se ejecuta esta celda para crear los DataFrames iniciales. La semilla (`SEED`) asegura que los datos generados sean siempre los mismos, garantizando la reproducibilidad del taller.


In [0]:
# Script para generar un conjunto de datos de e-commerce enriquecido
from pyspark.sql import SparkSession
from faker import Faker
import random
from datetime import datetime, timedelta

# --- SEMILLA PARA REPRODUCIBILIDAD ---
# Garantiza que los datos generados sean siempre los mismos en cada ejecución.
SEED = 2025
Faker.seed(SEED)
random.seed(SEED)

# Inicializar Faker para generar datos en español
fake = Faker('es_ES')

# --- DEFINICIÓN DE FUNCIONES PARA GENERAR DATOS ---
def generar_usuarios(n=1000):
    """Genera una lista de diccionarios de usuarios."""
    return [{'id_usuario': 1000 + i, 'nombre': fake.name(), 'email': fake.email(), 'fecha_registro': fake.date_time_between(start_date='-2y'), 'ciudad': fake.city()} for i in range(n)]

def generar_productos(n=500):
    """Genera una lista de productos."""
    categorias = ['Electrónica', 'Hogar', 'Ropa', 'Libros', 'Deportes', 'Juguetes', 'Alimentos']
    data = []
    for i in range(n):
        data.append({
            'id_producto': 2000 + i,
            'nombre_producto': fake.word().capitalize() + " " + fake.word(),
            'categoria': random.choice(categorias),
            'precio_unitario': round(random.uniform(5.0, 350.0), 2) # Ojo: PySpark inferirá esto como DoubleType
        })
    return data

def generar_pedidos(usuarios, productos, n=10000):
    """Genera una lista de pedidos, vinculando usuarios y productos."""
    data = []
    for i in range(n):
        usuario = random.choice(usuarios)
        producto = random.choice(productos)
        cantidad = random.randint(1, 5)
        data.append({
            'id_pedido': 3000 + i,
            'id_usuario': usuario['id_usuario'],
            'id_producto': producto['id_producto'],
            'cantidad': cantidad,
            'monto': round(cantidad * producto['precio_unitario'], 2) # Inferido como DoubleType
        })
    return data

# --- CREACIÓN DE DATAFRAMES EN MEMORIA ---
print("Generando DataFrames en memoria...")
usuarios_data = generar_usuarios()
productos_data = generar_productos()
pedidos_data = generar_pedidos(usuarios_data, productos_data)

usuarios_df = spark.createDataFrame(usuarios_data)
productos_df = spark.createDataFrame(productos_data)
pedidos_df = spark.createDataFrame(pedidos_data)

print("DataFrames generados exitosamente.")

### **Ingesta a la Capa Bronce y el Error Común de Esquema**

Ahora, se persistirán estos datos crudos en tablas Delta, nuestra capa **Bronce**.

Aquí es donde a menudo surge el error `DELTA_FAILED_TO_MERGE_FIELDS`. Este error ocurre si se intenta sobrescribir una tabla Delta con datos que tienen un tipo de dato diferente en una columna. Por ejemplo, si PySpark infiere `precio_unitario` como `DOUBLE` la primera vez, y en una ejecución posterior, un `CAST` explícito intenta escribirlo como `DECIMAL`, Delta arrojará este error para proteger la integridad del esquema.

**La Solución:** Para permitir que Delta actualice el esquema de la tabla de forma segura, se utiliza la opción `mergeSchema`.

-----

### **Persistencia en la Capa Bronce con Evolución de Esquema**

**Acción:** Se guardan los DataFrames como tablas Delta. Se añade `.option("mergeSchema", "true")` para evitar errores de esquema si el cuaderno se ejecuta varias veces con ligeros cambios en los tipos de datos.

In [0]:
# Persistiendo los DataFrames como Tablas Delta en la Capa Bronce
db_name = "curso_arquitecturas"
spark.sql(f"CREATE DATABASE IF NOT EXISTS {db_name}")
spark.sql(f"USE {db_name}")

print(f"Usando la base de datos: '{db_name}'")

# Se usa .option("mergeSchema", "true") como buena práctica para la evolución del esquema
usuarios_df.write.format("delta").option("mergeSchema", "true").mode("overwrite").saveAsTable("usuarios_bronze")
productos_df.write.format("delta").option("mergeSchema", "true").mode("overwrite").saveAsTable("productos_bronze")
pedidos_df.write.format("delta").option("mergeSchema", "true").mode("overwrite").saveAsTable("pedidos_bronze")

print("Tablas Bronze creadas/actualizadas exitosamente.")

### **Transformación a la Capa Plata (Silver)**

**Acción:** Con los datos crudos en la capa Bronce, se procede a limpiarlos y estructurarlos en la capa **Plata**. Aquí se realizan conversiones de tipo explícitas (`CAST`) para asegurar la calidad y consistencia de los datos para el análisis.


In [0]:
%sql
-- Creación de las tablas Silver a partir de Bronze
USE curso_arquitecturas;

CREATE OR REPLACE TABLE usuarios_silver AS
SELECT 
  id_usuario, 
  nombre AS nombre_usuario, 
  email, 
  fecha_registro, 
  ciudad 
FROM usuarios_bronze 
WHERE id_usuario IS NOT NULL;

CREATE OR REPLACE TABLE productos_silver AS
SELECT 
  id_producto, 
  nombre_producto, 
  categoria, 
  CAST(precio_unitario AS DECIMAL(10, 2)) as precio_unitario -- Se define un tipo de dato explícito y correcto
FROM productos_bronze 
WHERE id_producto IS NOT NULL;

CREATE OR REPLACE TABLE pedidos_silver AS
SELECT 
  id_pedido, 
  id_usuario, 
  id_producto, 
  cantidad, 
  CAST(monto AS DECIMAL(18, 2)) AS monto_total, -- Se define un tipo de dato explícito
  fecha_pedido AS ts_pedido
FROM pedidos_bronze 
WHERE id_pedido IS NOT NULL;

### **El Problema de Negocio: Clientes de Alto Valor**

**Contexto:** El equipo de marketing necesita identificar a los **"clientes de alto valor"**, definidos como aquellos cuyo gasto total está por encima del gasto promedio de todos los clientes.

-----

### **La "Mala" Solución: El Infierno de las Subconsultas 👹**

**Análisis:** Este es un enfoque común para resolver el problema, pero anida la lógica de manera que se vuelve difícil de leer, depurar y, lo que es peor, repite código.



In [0]:
%sql
-- La "Mala" Solución con subconsultas anidadas
SELECT
  u.id_usuario,
  u.nombre_usuario,
  TotalGastadoPorCliente.GastoTotal
FROM
  usuarios_silver u
  JOIN (
    -- Subconsulta 1: Calcula el gasto total de cada cliente.
    SELECT id_usuario, SUM(monto_total) AS GastoTotal
    FROM pedidos_silver
    GROUP BY id_usuario
  ) AS TotalGastadoPorCliente ON u.id_usuario = TotalGastadoPorCliente.id_usuario
WHERE
  TotalGastadoPorCliente.GastoTotal > (
    -- Subconsulta 2: Calcula el gasto promedio... ¡repitiendo la lógica interna!
    SELECT AVG(GastoTotal)
    FROM (
        SELECT SUM(monto_total) AS GastoTotal
        FROM pedidos_silver
        GROUP BY id_usuario
    ) AS GastoPromedio
  );

### **¿Por Qué es una "Mala" Solución?**

El código anterior es ineficiente y propenso a errores por tres razones principales:

1.  **Ilegibilidad:** La lógica no es secuencial. Para entenderla, es necesario "desenrollar" las consultas desde adentro hacia afuera.
2.  **Difícil de Depurar:** Aislar y probar una de las subconsultas internas requiere copiar y pegar, lo cual es tedioso.
3.  **Repetición de Código (No es DRY):** La lógica para calcular el gasto total por cliente se escribe dos veces. Un cambio en la definición de "gasto" requeriría modificar dos partes del código.

-----

### **La "Buena" Solución: Dominando los CTEs ✨**

**Análisis:** Se refactoriza la consulta usando CTEs. La cláusula `WITH` introduce bloques de construcción lógicos y nombrados, haciendo que la consulta se lea como una receta.

In [0]:
%sql
-- La "Buena" Solución con CTEs
WITH GastoPorCliente AS (
  -- Paso 1: Se calcula el gasto total por cliente y se nombra el resultado "GastoPorCliente".
  SELECT
    id_usuario,
    SUM(monto_total) AS GastoTotal
  FROM pedidos_silver
  GROUP BY id_usuario
),
PromedioGeneral AS (
  -- Paso 2: Se reutiliza el CTE anterior para calcular el promedio general. Sin repetir código.
  SELECT AVG(GastoTotal) AS GastoPromedio
  FROM GastoPorCliente
)
-- Paso 3: La consulta final es ahora simple y clara.
SELECT
  u.id_usuario,
  u.nombre_usuario,
  gpc.GastoTotal
FROM usuarios_silver u
JOIN GastoPorCliente gpc ON u.id_usuario = gpc.id_usuario
-- Se cruza con el promedio para filtrar
CROSS JOIN PromedioGeneral
WHERE gpc.GastoTotal > PromedioGeneral.GastoPromedio
ORDER BY gpc.GastoTotal DESC;

### **Anatomía de un Common Table Expression**

La sintaxis de un CTE es simple y poderosa. Es una de las herramientas más importantes para organizar el código SQL.

```sql
WITH 
  nombre_cte_1 AS (
    -- SELECT que define el primer bloque lógico
  ),
  nombre_cte_2 AS (
    -- SELECT que define el segundo bloque, puede usar el primero
  )
-- SELECT principal que usa los CTEs como si fueran tablas
SELECT ... FROM nombre_cte_1 JOIN nombre_cte_2 ON ...;
```

  * **`WITH`**: Inicia la declaración de CTEs.
  * **`nombre_cte AS (...)`**: Define un resultado temporal con un nombre.
  * **Coma (`,`)**: Separa las definiciones de múltiples CTEs.
  * **Consulta Principal**: La sentencia `SELECT` final que orquesta los resultados de los CTEs.

-----

### **El Flujo de Datos en un CTE**

Visualmente, se puede pensar en los CTEs como una tubería de datos. Cada CTE toma datos, los procesa y pasa el resultado al siguiente paso, hasta llegar a la consulta final.

**Datos Crudos (`pedidos_silver`)** → **CTE `GastoPorCliente`** → **CTE `PromedioGeneral`** → **Resultado Final**

Esta estructura secuencial es la razón por la que el código se vuelve tan intuitivo.

-----

### **Ejemplo Adicional: CTEs con JOINs Internos**

Un CTE no es solo para agregaciones. Puede contener cualquier lógica, incluyendo `JOINs`.

**Problema:** Se quiere saber el total de ventas por **categoría de producto**.



In [0]:
%sql
WITH VentasConCategoria AS (
  -- En este CTE, se unen pedidos y productos para enriquecer cada venta con su categoría.
  SELECT
    p.monto_total,
    pr.categoria
  FROM
    pedidos_silver p
    JOIN productos_silver pr ON p.id_producto = pr.id_producto
)
-- La consulta final es ahora muy simple, porque el trabajo complejo se hizo en el CTE.
SELECT
  categoria,
  SUM(monto_total) AS VentasTotales,
  COUNT(*) AS NumeroDePedidos
FROM
  VentasConCategoria
GROUP BY
  categoria
ORDER BY
  VentasTotales DESC;

### **Momento de Práctica 🏋️**

Ahora es el turno del participante. Se resolverá un nuevo problema de negocio usando lo aprendido.

**Problema:** Se necesita un reporte que muestre, para cada **ciudad**, el número de **clientes únicos** que han realizado pedidos y el **monto total** gastado por los clientes de esa ciudad.

**Requerimiento:** Escribir una consulta usando CTEs que devuelva `ciudad`, `numero_clientes` y `gasto_total_ciudad`.

**Pistas:**

1.  Se necesitará unir `pedidos_silver` y `usuarios_silver`.
2.  Un buen primer CTE podría agregar los pedidos para obtener el gasto total por `id_usuario`.
3.  La consulta final puede unir ese resultado con `usuarios_silver` para agrupar por `ciudad`.

-----

### **Espacio para la Solución del Participante**


In [0]:
%sql
-- El participante escribe su solución aquí...

### **¿Cuándo NO usar un CTE?**

Aunque los CTEs son fantásticos, no son una solución universal. Hay un punto importante a considerar:

**Optimización:** Un CTE se re-calcula cada vez que se referencia en la consulta principal. Si se tiene un CTE muy costoso y se llama múltiples veces, su lógica se ejecutará múltiples veces. En esos casos, una **vista temporal (`CREATE TEMP VIEW`)** puede ser más eficiente, ya que materializa el resultado una vez y lo reutiliza.


In [0]:
%sql
-- Ejemplo de Vista Temporal (Alternativa para CTEs costosos y reutilizados)

-- Paso 1: Se materializa el resultado. Esto se calcula UNA SOLA VEZ.
CREATE OR REPLACE TEMP VIEW GastoPorCliente_temp AS
SELECT
  id_usuario,
  SUM(monto_total) AS GastoTotal
FROM
  pedidos_silver
GROUP BY
  id_usuario;

-- Paso 2: Ahora se puede usar la vista temporal como una tabla.
-- Si se usara en múltiples JOINs, el cálculo no se repetiría.
SELECT * FROM GastoPorCliente_temp LIMIT 10;


### **Resumen del Módulo**

Se ha completado el módulo. El participante ahora cuenta con una herramienta fundamental en su arsenal de SQL.

**Ideas Clave:**

  * **CTEs (`WITH ... AS`)** transforman subconsultas anidadas en bloques de código secuenciales y legibles.
  * **Mejoran la Mantenibilidad:** Facilitan la lectura, depuración y modificación del código.
  * **Promueven la Lógica Paso a Paso:** Permiten construir consultas complejas como si fueran una receta.
  * **No son una Panacea:** Para lógica muy costosa que se reutiliza varias veces, una `TEMP VIEW` puede ser más performante.

-----

### **Próximos Pasos**

En el siguiente módulo, **"Más Allá del `GROUP BY`: Desbloqueando Análisis con Funciones de Ventana"**, se llevará el análisis a otro nivel, aprendiendo a realizar cálculos sobre conjuntos de filas, como rankings, diferencias temporales y promedios móviles, sin necesidad de complejos `self-joins`.

---

### **Solución del Ejercicio de Ciudades**

Aquí se presenta una forma de resolver el problema.


In [0]:
%sql
---- Solución del Ejercicio de Ciudades

WITH GastoPorCliente AS (
  -- Paso 1: Se calcula el gasto total por cada cliente.
  SELECT
    id_usuario,
    SUM(monto_total) AS GastoTotal
  FROM
    pedidos_silver
  GROUP BY
    id_usuario
)
-- Paso 2: Se unen los gastos con la información de los usuarios y se agrupa por ciudad.
SELECT
  u.ciudad,
  COUNT(DISTINCT u.id_usuario) AS numero_clientes,
  SUM(gpc.GastoTotal) AS gasto_total_ciudad
FROM
  usuarios_silver u
  JOIN GastoPorCliente gpc ON u.id_usuario = gpc.id_usuario
GROUP BY
  u.ciudad
ORDER BY
  gasto_total_ciudad DESC;